In [ ]:
"""Aircraft cost models"""
from gpkit.shortcuts import Var, Model
import gpkit
gpkit.disable_units()

class DAPCA4Cost(Model):
    """
    Cost Model from Raymer, Aircraft Design: A Conceptual Approach 3rd Edition,
    pp 713-714
    Note: this is also one instance of the RAND Corp cost models.
    Cost is in 2012 dollars
    Units disabled because '$' is not defined as a unit 
    
    Labor Rates are found from Raymer

    Default values based on Honda Jet specs

    Arguments
    ---------
    engine_defined : bool
        If true, must specify T_max, M_max, and T_tur
        If false, must define engine_cost
        Todo: this should really be replaced with an engine cost (sub) model.
    """
    def setup(self, W_e=9200, V=420, Q=5, FTA=1, N_eng=10, T_max=2050,
              M_max=0.63, T_tur=2050, R_avn=5000, engine_cost=500000,
              eng_defined=False):

        # User Definied Variables
        W_e = Var('W_e', W_e, "lb", "Empty Weight")
        V = Var('V', V, "knots", "Maximum Velocity")
        Q = Var('Q', Q, "-", "Number produced in 5 years")
        FTA = Var('FTA', FTA, "-", "Number of Flight Test Aircraft")
        N_eng = Var('N_{eng}', N_eng, "-",
                    "Number of engines or Q*num of engines per aircraft")
        T_max = Var('T_{max}', T_max, "lb", "Maximum Thrust")
        M_max = Var('M_{max}', M_max, "-", "Maximum Mach # of engine")
        T_tur = Var('T_{tur}', T_tur, "R", "Turbine Inlet Temperature")
        R_avn = Var('C_{avn}', R_avn, "$/lb", "Avionics Cost")

        # Constants Hourly Rates
        R_E = Var('R_{E}', 115, "$/hr", "Enginering Hourly Rate")
        R_T = Var('R_{T}', 118, "$/hr", "Tooling Hourly Rate")
        R_M = Var('R_{M}', 108, "$/hr", "Manufacturing Hourly Rate")
        R_Q = Var('R_{Q}', 98, "$/hr", "Quality Check Hourly Rate")

        # Free Variables

        # Hourly costs in hrs. Multiply by hourly work rate to get cost
        H_E = Var('H_{E}', "hrs",
                  "Engineering hours of airframe and component integration")
        #Eng Hours does not include cost of avionics or engine engineering
        H_T = Var('H_{T}', "hrs", "Tooling hours for production preparation")
        #H_T also covers tooling cost through ongoing production
        H_M = Var('H_{M}', "hrs",
                  "Manufacturing hours of main and subcontractors")
        H_Q = Var('H_{Q}', "hrs", "Quality control hours for inspection")

        # Cost Varibles
        C_D = Var('C_D', "$", "Development Cost")
        C_F = Var('C_F', "$", "Flight Test Cost to prove airworthiness")
        C_M = Var('C_M', "$", "Materials cost of aluminum airframe")
        C_fly = Var('C_{tot}', "$", "RDT&E Flyaway Cost")
        C_avn = Var('C_{avn}', "$", "Cost of avionics")
        #Raymer suggests C_avn = 0.15*C_fly

        eng = (3112*(0.043*T_max + 243.35*M_max + 0.969*T_tur - 2228)
               if eng_defined else engine_cost)
        C_eng = Var('C_{eng}', eng, '$', "Engine Cost")

        objective = C_fly
        constraints = [C_fly >= (H_E*R_E + H_T*R_T + H_M*R_M + H_Q*R_Q + C_D +
                                 C_F + C_M + C_eng*N_eng + C_avn),
                       H_E >= 4.86 * W_e**0.777 * V**0.894 * Q**0.163,
                       H_T >= 5.99 * W_e**0.777 * V**0.696 * Q**0.263,
                       H_M >= 7.37 * W_e**0.82 * V**0.484 * Q**0.641,
                       H_Q >= 0.133*H_E,
                       C_D >= 91.3 * W_e**0.630 * V**1.3,
                       C_F >= 2498 * W_e**0.325 * V**0.822 * FTA**1.21,
                       C_M >= 22.1 * W_e**0.921 * V**0.621 * Q**0.799,
                       C_avn >= R_avn*W_e,
                      ]
        return objective, constraints

    def test(self):
        sol = self.solve()
        H_E = sol('H_{E}')
        H_T = sol('H_{T}')
        H_M = sol('H_{M}')
        H_Q = sol('H_{Q}')
        R_E = sol('R_{E}')
        R_T = sol('R_{T}')
        R_M = sol('R_{M}')
        R_Q = sol('R_{Q}')
        C_D = sol('C_{D}')
        C_F = sol('C_{F}')
        C_M = sol('C_{M}')
        C_avn = sol('C_{avn}')
        C_fly = sol('C_{fly}')


In [ ]:
"""Breguet range model"""
from gpkit.shortcuts import Var, Model
from gpkit.tools import te_exp_minus1


class BreguetRange(Model):

    """Breguet Range Model

    Assumptions
    -----------
    Fuel burn varies linearily with thrust and is independent of velocity.
    """
    def setup(self):
        
        # insert cost model
        cost = DAPCA4Cost()
        V = cost["V"]

        TSFC_min = Var('TSFC_{min}', 0.307, "lb/lbf/hr", "Minimum TSFC")
        MTOW = Var('MTOW', 10000, "lbf", "Max take off weight")
        W_e = Var('W_{e}', 7000, "lbf", "Operating Empty Weight")
        LoverD_max = Var('LoverD_{max}', 15, "-", "Maximum Lift to Drag Ratio")
        M_max = Var('M_max', 0.78, "-", "Maximum Mach")

        #Constants
        g = Var('g', 9.81, "m/s^2", "gravity")
        a0 = Var('a0', 340.29, "m/s", "speed of sound at sea level")

        #Free Variables
        R = Var('R', "nautical_miles", "range")
        M = Var('M', "-", "Mach number")
        LoverD = Var('LoverD', "-", "life to drag ratio")
        TSFC = Var('TSFC', "lb/lbf/hr", "thrust specific fuel consuption")
        W_init = Var('W_init', "lbf", "initial weight")
        W_fuel = Var('W_fuel', "lbf", "fuel weight")
        z_bre = Var('z_{bre}', "-", "Breguet parameter")
        t = Var('t', "hr", "time")

        # Set up Model Equations
        objective = 1/R  # Maximize range
        constraints = [W_init >= W_e + W_fuel,
                       W_init <= MTOW,
                       LoverD <= LoverD_max,
                       TSFC >= TSFC_min,
                       M <= V/a0,
                       t >= R/M/a0,
                       z_bre >= t*TSFC*g/LoverD,
                       W_fuel/W_e >= te_exp_minus1(z_bre, nterm=3)
                       ]
        
        model = Model(objective, constraints)
        
        model = model & cost
        
        return model
    
    def test(self):
        self.solve(verbosity=0)
                
print BreguetRange().solve()
